Autor: ADONIS DARIO PIRES GOMES DE MORAIS

In [19]:
import pandas as pd
import numpy as np
import re

# Funções utilizadas

In [20]:
# 2003: Preliminar com mini grupos
# 2004-07: Preliminar com ida e volta com Brasileiros
# 2008-16: BRA na 2a fase Preliminar contra outros BRA
# 2017-20: BRA na 1a fase Preliminar contra SAM
# 2021-22: BRA na Fase GP
# 2023-24: BRA na Fase GP com PO p/ 2º colocado

# https://pt.wikipedia.org/wiki/Participa%C3%A7%C3%B5es_de_clubes_brasileiros_de_futebol_em_competi%C3%A7%C3%B5es_internacionais#Copa_Sul-Americana

In [21]:
# @title Criando o dataframe
# Inicializando os vetores que serão utilizados para receber os times nas respectivas fases
top_f3 = []
top_f1 = []
top_f2 = []
top_po = []
top_gp = []
campeao = []
vice = []
finalistas = []
participantes = []
# Colunas do Dataframe
colunas_df = ['Time', 'CON_B_1', 'CON_B_2', 'CON_B_SF', 'CON_B_QF', 'CON_B_OF', 'CON_B_PO', 'CON_B_GP', 'CON_B_F1', 'CON_B_F2', 'CON_B_F3', 'CON_B_P']
colunas_br = ['4-Fase', '3-Fase', '2-Fase', '1-Fase', 'Oitavas', 'Quartas', 'Semis', 'Final']
df_copa = pd.DataFrame(columns=colunas_df)
df_copa

,Time,CON_B_1,CON_B_2,CON_B_SF,CON_B_QF,CON_B_OF,CON_B_PO,CON_B_GP,CON_B_F1,CON_B_F2,CON_B_F3,CON_B_P


Dicionário de colunas

| Nome | Descrição |
|------|-----------|
| COP_1 | Vencedor da edição |
| COP_2 | Vice da Edição |
| COP_4 | Elim. na Semifinal |
| COP_8 | Elim. nas Quartas de final |
| COP_16 | Elim. nas Oitavas de final |
| COP_F1 | Elim. em 1 fase antes das Oitavas (-1F) |
| COP_F2 | Elim. em 2 fases antes das Oitavas (-2F) |
| COP_F3 | Elim. em 3 fases antes das Oitavas (-3F) |
| COP_F4 | Elim. em 4 fases antes das Oitavas (-4F) |
| COP_P | Número de participações |

In [22]:
# @title Função limpar nomes
# Usando regex para limpar o '(x)' no final do nome do time
def clean_text(text):
    text = pd.Series(text).replace(r'\[.*?\]', '', regex=True).replace(r'\(.*?\)', '', regex=True)
    # Remove espaços em branco no início e no final
    return text.str.strip()

In [23]:
# @title Função get Bracket
# Esta função pega o chaveamento da copa na Wikipedia
def get_bracket(final, bracket):
  top_sf = []
  top_qf = []
  top_of = []
  top_po = []
  top_gp = []
  top_f1 = []
  top_f2 = []
  top_f3 = []
  bracket = bracket.loc[2:, :]
  participantes = final
  print(f'Finalistas: {final}')

  # A partir do final do Chaveamento anda as colunas até a fase inicial para recuperar os times
  for i in range(len(bracket.columns)-1, 0, -1):

      # Pegar os times sem repetição na coluna desejada
      times = bracket.loc[:, i].dropna().unique()
      # Retira os possíveis () e []
      times = pd.Series(times).replace(r'\[.*?\]', '', regex=True).replace(r'\(.*?\)', '', regex=True)
      # Define o tipo como String
      times = times.astype(str)
      # Garante que o conteúdo não contém apenas números
      times = times[~times.str.match(r'^[0-9\s]+$')]
      times = times.to_list()
      times = [time.strip() for time in times]
      # Limpa o conteúdo
      times = [team.strip() for team in times if isinstance(team, str) and re.match(r'^(?!\d+$)[A-Za-zÀ-ÿ0-9\s\-\.\']+$', team)]
      # Se a quantidade de times retornada for igual a 4, quer dizer que a coluna é a semifinal

      if (len(times) == 4) & (len(top_sf) == 0):
          # Recupera os times que ainda não estão na lista 'participantes'
          top_sf = [team for team in times if team not in participantes]
          print(f'Semis:  {top_sf}')
          # Adiciona os times da semifinal à lista 'participantes'
          participantes = set(participantes).union(top_sf)

      elif (len(times) == 8) & (len(top_qf) == 0): # 8 times = coluna Quartas de final
          top_qf = [team for team in times if team not in participantes]
          print(f'Quartas: {top_qf}')
          participantes = set(participantes).union(top_qf)

      elif (len(times) > 8) & (len(top_of) == 0): # 16 times = coluna Oitavas de final
          top_of = [team for team in times if team not in participantes]
          print(f'Oitavas: {top_of}')
          participantes = set(participantes).union(top_of)


  return participantes, top_sf, top_qf, top_of

In [24]:
# @title Função get Table Malformatada
def get_table_unformatted(dataframe):
  # Remover colunas multinivel e colunas que não serão utilizadas
  dataframe = dataframe.drop(dataframe.columns[-2:], axis=1)
  dataframe.columns = dataframe.columns.droplevel(0)
  dataframe.columns = dataframe.columns.get_level_values(0)
  dataframe['Resultado'] = ""
  res = "Campeão"
  indices = []
  for index, row in dataframe.iterrows():
      time = row['Times']
      if 'campeão' in time or 'Eliminados' in time:
        res = time
        indices.append(index)
      else:
        dataframe.at[index, 'Resultado'] = res
  dataframe.drop(indices, inplace=True)
  return dataframe

In [25]:
# @title Função get times preliminares
# Esta função recupera os times que foram eliminados nas fases preliminares ou
# em uma fase cuja tabela tem as colunas: Equipe 1 | Resultado | Equipe 2
def get_times_preliminares(df, list_todos, casa=1, fora=3):
  df.rename(columns={df.columns[casa]: 'Equipe 1', df.columns[fora]: 'Equipe 2'}, inplace=True)
  # df.columns.values[1] = "Equipe 1"
  # df.columns.values[3] = "Equipe 2"
  df_pre = pd.concat([df['Equipe 1'], df['Equipe 2']], ignore_index=True)
  top = df_pre.astype(str)
  top = top.replace(r'\[.*?\]', '', regex=True).replace(r'\(.*?\)', '', regex=True)
  top = top[~top.str.match(r'^[0-9\s]+$')]
  top = top.to_list()
  top = [team.strip() for team in top
         if isinstance(team, str) and re.match(r'^(?!\d+$)[A-Za-zÀ-ÿ0-9\s\-]+$', team)
         and ("classificado") not in team
         and ("Classificação") not in team
         and ("classifica") not in team
         and ("Preliminar") not in team
         and team not in list_todos]
  return top

In [26]:
# @title Função get Times Grupos
def get_times_grupos(df, list_todos):
  df.columns.values[1] = "Equipe"
  times = [team.strip() for team in df['Equipe'].to_list()
           if isinstance(team, str) and re.match(r'^(?!\d+$)[A-Za-zÀ-ÿ0-9\s\-]+$', team)
           and team not in list_todos]
  return times

In [27]:
# @title Função get Table com Beautiful Soup
def get_table_bs(url, t):
  import requests
  from bs4 import BeautifulSoup

  wikiurl = url
  table_class = "wikitable sortable jquery-tablesorter"
  response = requests.get(wikiurl)
  # 200
  soup = BeautifulSoup(response.text, 'html.parser')
  tables_all = soup.find_all('table', {"class": "wikitable"})

  # <table class="wikitable sortable">
  # <tbody><tr>
  # <th>Name of Town
  # </th>
  # <th>State
  # ....

  df = pd.read_html(str(tables_all[t]))
  return df

In [28]:
# @title Função preencher tabela
def preencher_tabela(times_ed, df):
  times_tabela = df['Time'].to_list()
  # Itera sobre os participantes da edição
  for team in times_ed:
      # Insere o time caso seja a primeira participação
      if team not in times_tabela:
          print(f'Novo participante: {team}')
          df.loc[len(df.index)] = [team, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

      if team == finalistas[0]:
          df.loc[df['Time'] == team, 'CON_B_1'] += 1
      elif team == finalistas[1]:
          df.loc[df['Time'] == team, 'CON_B_2'] += 1
      elif team in top_sf:
          df.loc[df['Time'] == team, 'CON_B_SF'] += 1
      elif team in top_qf:
          df.loc[df['Time'] == team, 'CON_B_QF'] += 1
      elif team in top_of:
          df.loc[df['Time'] == team, 'CON_B_OF'] += 1
      elif team in top_po:
          df.loc[df['Time'] == team, 'CON_B_PO'] += 1
      elif team in top_gp:
          df.loc[df['Time'] == team, 'CON_B_GP'] += 1
      elif team in top_f1:
          df.loc[df['Time'] == team, 'CON_B_F1'] += 1
      elif team in top_f2:
          df.loc[df['Time'] == team, 'CON_B_F2'] += 1
      elif team in top_f3:
          df.loc[df['Time'] == team, 'CON_B_F3'] += 1

      # Adiciona a participação do time na edição
      df.loc[df['Time'] == team, 'CON_B_P'] += 1

  return df

# 1992-1999: Copa Conmebol

In [29]:
# 1992-96: f=0, b=3
# 1997: f=0, b=50
# 1998: f=0, b=43
# 1999: f=1, b=45
for i in range(1992, 1999+1):
  ano = str(i)
  df = pd.read_html('https://pt.wikipedia.org/wiki/Copa_Conmebol_de_'  + ano)
  print(f'Copa de {i}:')
  f=0
  b=3
  if (i == 1997):
    b=50
  elif (i == 1998):
    b=43
  elif (i == 1999):
    f=1
    b=45
  dftemp = df[f].T.reset_index(drop=True)
  dftemp.columns = dftemp.iloc[0]
  dftemp = dftemp.drop(0)
  dftemp = dftemp.drop(2)
  campeao = clean_text(dftemp['Campeão'])
  vice = clean_text(dftemp['Vice-campeão'])
  finalistas = campeao.to_list() + vice.to_list()
  participantes, top_sf, top_qf, top_of = get_bracket(finalistas, df[b])

  df_copa = preencher_tabela(participantes, df_copa)

Copa de 1992:
Finalistas: ['Atlético Mineiro', 'Olimpia']
Semis:  ['El Nacional', 'Gimnasia y Esgrima']
Quartas: ['Grêmio', 'Junior de Barranquilla', 'Deportivo Español', 'Peñarol']
Oitavas: ['Bragantino', 'Universitario', 'Marítimo', 'Fluminense', 'Oriente Petrolero', 'Vélez Sarsfield', 'Danubio', "O'Higgins"]
Novo participante: Olimpia
Novo participante: El Nacional
Novo participante: Deportivo Español
Novo participante: Danubio
Novo participante: Marítimo
Novo participante: Peñarol
Novo participante: Junior de Barranquilla
Novo participante: Fluminense
Novo participante: Grêmio
Novo participante: O'Higgins
Novo participante: Vélez Sarsfield
Novo participante: Universitario
Novo participante: Oriente Petrolero
Novo participante: Gimnasia y Esgrima
Novo participante: Atlético Mineiro
Novo participante: Bragantino
Copa de 1993:
Finalistas: ['Botafogo', 'Peñarol']
Semis:  ['San Lorenzo', 'Atlético Mineiro']
Quartas: ['Sportivo Luqueño', 'Colo-Colo', 'Deportivo Sipesa', 'Caracas']
Oitava

# 2003: BRA x BRA 2 Preliminares

In [30]:
df = pd.read_html('https://pt.wikipedia.org/wiki/Copa_Sul-Americana_de_2003')
df[4]

,Preliminar brasileira I,Preliminar brasileira I.1,Preliminar brasileira I.2
0,Santos,1 - 1,Internacional
1,Internacional,3 - 1,Flamengo
2,Flamengo,0 - 3,Santos
3,Preliminar brasileira II,Preliminar brasileira II,Preliminar brasileira II
4,Corinthians,0 - 2,Atlético Mineiro
5,Fluminense,2 - 0,Corinthians
6,Atlético Mineiro,0 - 2,Fluminense
7,Preliminar brasileira III,Preliminar brasileira III,Preliminar brasileira III
8,Grêmio,0 - 4,São Paulo
9,Vasco da Gama,1 - 1,Grêmio


In [31]:
# 2003: f=0, p1=4, p2=5, b=9
for i in range(2003, 2003+1):
  ano = str(i)
  df = pd.read_html('https://pt.wikipedia.org/wiki/Copa_Sul-Americana_de_'  + ano)
  print(f'Copa de {i}:')
  f=0
  p1=4
  p2=5
  b=9
  dftemp = df[f].T.reset_index(drop=True)
  dftemp.columns = dftemp.iloc[0]
  dftemp = dftemp.drop(0)
  dftemp = dftemp.drop(2)
  campeao = clean_text(dftemp['Campeão'])
  vice = clean_text(dftemp['Vice-campeão'])
  finalistas = campeao.to_list() + vice.to_list()
  participantes, top_sf, top_qf, top_of = get_bracket(finalistas, df[b])

  top_f1 = get_times_preliminares(df[p2], participantes, 0, 2)
  participantes = set(participantes).union(top_f1)
  print(f'2ª Fase: {top_f1}')

  df_2f = df[p1]
  df_2f.columns = ['Equipe 1', 'Resultado', 'Equipe 2']

  # top_f2 = get_times_preliminares(df_2f, participantes, 0, 2)
  top_f2 = df_2f['Equipe 1'].astype(str)
  top_f2 = top_f2.replace(r'\[.*?\]', '', regex=True).replace(r'\(.*?\)', '', regex=True)
  top_f2 = top_f2[~top_f2.str.match(r'^[0-9\s]+$')]
  top_f2 = top_f2.to_list()
  top_f2 = [team.strip() for team in top_f2
         if isinstance(team, str) and re.match(r'^(?!\d+$)[A-Za-zÀ-ÿ0-9\s\-]+$', team)
         and ("classificado") not in team
         and ("Classificação") not in team
         and ("classifica") not in team
         and ("Preliminar") not in team
         and team not in participantes]
  participantes = set(participantes).union(top_f2)
  print(f'1ª Fase: {top_f2}')


  df_copa = preencher_tabela(participantes, df_copa)

Copa de 2003:
Finalistas: ['Cienciano', 'River Plate']
Semis:  ['Atlético Nacional', 'São Paulo']
Quartas: ['Santos', 'Boca Juniors', 'The Strongest', 'Libertad']
2ª Fase: ['Independiente', 'Colón', 'São Caetano', 'LDU Quito', 'Nacional', 'Fluminense', 'San Lorenzo', 'Universidad Católica']
1ª Fase: ['Internacional', 'Flamengo', 'Corinthians', 'Atlético Mineiro', 'Grêmio', 'Vasco da Gama', 'Palmeiras', 'Cruzeiro']
Novo participante: Libertad
Novo participante: Independiente
Novo participante: Universidad Católica
Novo participante: Atlético Nacional
Novo participante: Cienciano
Novo participante: Nacional
Novo participante: Cruzeiro
Novo participante: Internacional
Novo participante: São Caetano
Novo participante: Boca Juniors
Novo participante: Flamengo


# 2004: BRA x BRA 3 Preliminares

In [32]:
# 2004: f=0, p1=6+3, p2=10+3, p3=14+1, b=30
for i in range(2004, 2004+1):
  ano = str(i)
  df = pd.read_html('https://pt.wikipedia.org/wiki/Copa_Sul-Americana_de_'  + ano)
  print(f'Copa de {i}:')
  f=0
  p1=6
  p2=10
  p3=14
  b=29
  dftemp = df[f].T.reset_index(drop=True)
  dftemp.columns = dftemp.iloc[0]
  dftemp = dftemp.drop(0)
  dftemp = dftemp.drop(2)
  campeao = clean_text(dftemp['Campeão'])
  vice = clean_text(dftemp['Vice-campeão'])
  finalistas = campeao.to_list() + vice.to_list()
  participantes, top_sf, top_qf, top_of = get_bracket(finalistas, df[b])
  df_1f = pd.DataFrame(columns=df[p2].columns)
  df_2f = df_1f.copy()
  df_3f = df_1f.copy()
  for i in range(0, 2):
    df_p = df[p3+i]
    df_p = df_p.drop(2)
    df_p = df_p.drop(1)
    df_1f = pd.concat([df_1f, df_p], ignore_index=True)

  top_f1 = get_times_preliminares(df_1f, participantes, 2, 4)
  participantes = set(participantes).union(top_f1)
  print(f'3ª Fase: {top_f1}')

  for i in range(0, 4):
    df_p = df[p2+i]
    df_p = df_p.drop(2)
    df_p = df_p.drop(1)
    df_2f = pd.concat([df_2f, df_p], ignore_index=True)

  top_f2 = get_times_preliminares(df_2f, participantes, 2, 4)
  participantes = set(participantes).union(top_f2)
  print(f'2ª Fase: {top_f2}')

  for i in range(0, 4):
    df_p = df[p1+i]
    df_p = df_p.drop(2)
    df_p = df_p.drop(1)
    df_3f = pd.concat([df_3f, df_p], ignore_index=True)

  top_f3 = get_times_preliminares(df_3f, participantes, 2, 4)
  participantes = set(participantes).union(top_f3)
  print(f'1ª Fase: {top_f3}')

  df_copa = preencher_tabela(participantes, df_copa)

Copa de 2004:
Finalistas: ['Boca Juniors', 'Bolívar']
Semis:  ['Internacional', 'LDU Quito']
Quartas: ['Junior', 'Cerro Porteño', 'Arsenal de Sarandí', 'Santos']
3ª Fase: ['São Paulo', 'Cruzeiro']
2ª Fase: ['Goiás', 'São Caetano', 'Flamengo', 'Grêmio']
1ª Fase: ['Paraná', 'Figueirense', 'Coritiba', 'Atlético Mineiro']
Novo participante: Coritiba
Novo participante: Junior
Novo participante: Arsenal de Sarandí
Novo participante: Cerro Porteño
Novo participante: Goiás
Novo participante: Figueirense


# 2005-16: 34-47 times - BRA x BRA na 2a Preliminar

In [33]:
df = pd.read_html('https://pt.wikipedia.org/wiki/Copa_Sul-Americana_de_2014')
df[6]

,Chave,Equipe 1,Total,Equipe 2,Ida,Volta
0,O1,Sport,1–3,Vitória,0–1,1–2
1,O2,Deportivo Capiatá,4–2,Caracas,1–1,3–1
2,O3,Godoy Cruz,0–3,River Plate,0–1,0–2
3,O4,Huachipato,2–1,Universidad Católica,2–0,0–1
4,O5,Fluminense,2–2 (gf),Goiás,2–1,0–1
5,O6,Peñarol,3–2,Deportivo Cali,2–2,1–0
6,O7,Lanús diretamente classificado às oitavas de f...,Lanús diretamente classificado às oitavas de f...,Lanús diretamente classificado às oitavas de f...,Lanús diretamente classificado às oitavas de f...,Lanús diretamente classificado às oitavas de f...
7,O8,Universitario de Sucre,2–5,Universidad César Vallejo,2–2,0–3
8,O9,Internacional,1–3,Bahia,0–2,1–1
9,O10,Independiente del Valle,1–3,Cerro Porteño,1–0,0–3


In [34]:
# 2005: f=0, p=15+3, b=21
# 2006: f=0, p=4, b=5
# 2007: f=1, p=5, b=12, drop("classificação")
# 2008: f=1, p=5, b=6
# 2009: f=0, p=4, b=5, drop("classificado")
# 2010-12, 14-16: f=0, p=5, b=6, drop("classificado")
# 2013: f=1, p=6, b=7, drop("classificado")
# 2014: f=1, p=6, b=7, drop("classificado")
for i in range(2005, 2016+1):
  ano = str(i)
  df = pd.read_html('https://pt.wikipedia.org/wiki/Copa_Sul-Americana_de_'  + ano)
  print(f'\nCopa de {i}:')
  f=0
  p=5
  if ((i == 2008) | (i == 2010) | (i == 2011) | (i == 2014)):
    f=1
  if (i == 2006) | (i == 2009):
    p-=1
  elif (i >= 2013) & (i <= 2015):
    f=1
    p+=1
  b=p+1
  if (i == 2007):
    f=1
    b=12
  if (i == 2005):
    p=15
    b=21
  dftemp = df[f].T.reset_index(drop=True)
  dftemp.columns = dftemp.iloc[0]
  dftemp = dftemp.drop(0)
  dftemp = dftemp.drop(2)
  campeao = clean_text(dftemp['Campeão'])
  vice = clean_text(dftemp['Vice-campeão'])
  finalistas = campeao.to_list() + vice.to_list()
  participantes, top_sf, top_qf, top_of = get_bracket(finalistas, df[b])
  df_pre = df[p]
  if (i == 2007):
    df_pre.columns = df_pre.columns.droplevel(0)
    df_pre.columns=['Equipe 1', 'Resultado', 'Equipe 2']
    top_f1 = get_times_preliminares(df_pre, participantes, 0, 2)
  else:
    if (i == 2005):
      df_pre = pd.DataFrame(columns=['Data', 'Equipe 1', 'Resultado', 'Equipe 2'])
      for i in range(0, 4):
        df_po = df[p+i]
        df_po.columns = df_po.iloc[0]
        df_po = df_po.drop(0)
        df_po = df_po.drop(1).reset_index(drop=True)
        df_po.columns.values[1] = "Equipe 1"
        df_po.columns.values[3] = "Equipe 2"
        df_pre = pd.concat([df_pre, df_po], ignore_index=True)

    top_f1 = get_times_preliminares(df_pre, participantes)
  participantes = set(participantes).union(top_f1)
  print(f'-1 Fase: {top_f1}')

  df_copa = preencher_tabela(participantes, df_copa)


Copa de 2005:
Finalistas: ['Boca Juniors', 'Pumas']
Semis:  ['Vélez Sarsfield', 'Universidad Católica']
Quartas: ['América', 'Corinthians', 'Fluminense', 'Internacional']
Oitavas: ['Atlético Nacional', 'Cruzeiro', 'River Plate', 'The Strongest', 'Banfield', 'D.C. United', 'Rosário Central', 'Cerro Porteño']
-1 Fase: ['Santos', 'São Paulo', 'Juventude', 'Goiás']
Novo participante: D.C. United
Novo participante: Banfield
Novo participante: América
Novo participante: Pumas
Novo participante: Juventude

Copa de 2006:
Finalistas: ['Pachuca', 'Colo-Colo']
Semis:  ['Atlético Paranaense', 'Toluca']
Quartas: ['Nacional', 'Lanús', 'Gimnasia y Esgrima', 'San Lorenzo']
Oitavas: ['Boca Juniors', 'River Plate', 'Corinthians', 'Deportes Tolima', 'Alajuelense', 'Fluminense', 'Santos', 'El Nacional']
-1 Fase: ['Libertad', 'Universitario de Sucre', 'Coronel Bolognesi', 'Paraná', 'Vasco da Gama', 'Botafogo', 'Mineros de Guayana', 'Vélez Sarsfield', 'Banfield', 'Cruzeiro']
Novo participante: Coronel Bolo

# 2017-20: 54 times - BRA x SAM 2 Preliminares

In [35]:
# 2017: f=0, p1=5, p2=9, b=10
# 2018: f=0, p1=6, p2=10, b=11
# 2019: f=1, p1=7, p2=11, b=12
# 2020: f=0, p1=6, p2=10, b=11
for i in range(2017, 2020+1):
  ano = str(i)
  df = pd.read_html('https://pt.wikipedia.org/wiki/Copa_Sul-Americana_de_'  + ano)
  print(f'Copa de {i}:')
  f = 0
  p=6
  b=11
  if (i == 2017):
    p-=1
    b-=1
  elif (i == 2019):
    f=1
    p+=1
    b+=1
  dftemp = df[f].T.reset_index(drop=True)
  dftemp.columns = dftemp.iloc[0]
  dftemp = dftemp.drop(0)
  dftemp = dftemp.drop(2)
  campeao = clean_text(dftemp['Campeão'])
  vice = clean_text(dftemp['Vice-campeão'])
  finalistas = campeao.to_list() + vice.to_list()
  participantes, top_sf, top_qf, top_of = get_bracket(finalistas, df[b])
  top_f1 = get_times_preliminares(df[p+4], participantes)
  participantes = set(participantes).union(top_f1)
  print(f'2ª Fase: {top_f1}')
  top_f2 = get_times_preliminares(df[p], participantes)
  participantes = set(participantes).union(top_f2)
  print(f'1ª Fase: {top_f2}')

  df_copa = preencher_tabela(participantes, df_copa)

Copa de 2017:
Finalistas: ['Independiente', 'Flamengo']
Semis:  ['Libertad', 'Junior de Barranquilla']
Quartas: ['Racing', 'Nacional', 'Fluminense', 'Sport']
Oitavas: ['Santa Fe', 'Corinthians', 'Estudiantes', 'Atlético Tucumán', 'LDU Quito', 'Chapecoense', 'Ponte Preta', 'Cerro Porteño']
2ª Fase: ['Deportivo Cali', 'Palestino', 'Nacional Potosí', 'Bolívar', 'Fuerza Amarilla', 'Huracán', 'Oriente Petrolero', 'Defensa y Justicia', 'Patriotas', 'Independiente Medellín', 'Deportes Iquique', 'Sol de América', 'Arsenal de Sarandí', 'Universidad Católica', 'Olimpia', 'Boston River']
1ª Fase: ['Petrolero', 'Everton', 'Estudiantes de Caracas', 'Deportivo Anzoátegui', 'Juan Aurich', 'Deportes Tolima', 'Cruzeiro', 'Sport Huancayo', 'Sportivo Luqueño', 'Defensor Sporting', 'Caracas', 'Deportivo Cuenca', 'Universidad de Chile', 'Alianza Lima', 'Gimnasia y Esgrima', 'Comerciantes Unidos', 'Atlético Venezuela', 'Danubio', 'Águilas Doradas', 'São Paulo', 'Liverpool']
Novo participante: Atlético Tucum

# 2021-22: 56 times - GP

In [36]:
# 2021: b=35, g=9+3
# 2022: b=36, g=10+3
for i in range(2021, 2022+1):

  ano = str(i)
  df = pd.read_html('https://pt.wikipedia.org/wiki/Copa_Sul-Americana_de_' + ano)
  print(f'Copa de {i}:')
  f=0
  b = 35
  g = 9
  if (i == 2022):
    b +=1
    g +=1
  dftemp = df[f].T.reset_index(drop=True)
  dftemp.columns = dftemp.iloc[0]
  dftemp = dftemp.drop(0)
  dftemp = dftemp.drop(2)
  campeao = clean_text(dftemp['Campeão'])
  vice = clean_text(dftemp['Vice-campeão'])
  finalistas = campeao.to_list() + vice.to_list()
  participantes, top_sf, top_qf, top_of = get_bracket(finalistas, df[b])

  top_gp = []
  for j in range(g, g+8*3, 3):
    grupo = get_times_grupos(df[j], participantes)
    #top_gp = set(top_gp).union(grupo)
    top_gp = top_gp + grupo

  participantes = set(participantes).union(top_gp)
  print(f'Grupos: {list(top_gp)}')

  df_copa = preencher_tabela(participantes, df_copa)

Copa de 2021:
Finalistas: ['Athletico Paranaense', 'Red Bull Bragantino']
Semis:  ['Peñarol', 'Libertad']
Quartas: ['Sporting Cristal', 'LDU Quito', 'Rosário Central', 'Santos']
Oitavas: ['Arsenal de Sarandí', 'Nacional', 'Grêmio', 'América de Cali', 'Deportivo Táchira', 'Independiente del Valle', 'Independiente', 'Junior de Barranquilla']
Grupos: ['Huachipato', 'San Lorenzo', '12 de Octubre', 'Montevideo City Torque', 'Bahia', 'Guabirá', 'Ceará', 'Bolívar', 'Jorge Wilstermann', 'Melgar', 'Aucas', 'Metropolitanos', 'Corinthians', 'River Plate-PAR', 'Sport Huancayo', 'Atlético Goianiense', 'Palestino', 'Emelec', 'Talleres', 'Deportes Tolima', 'Lanús', 'La Equidad', 'Aragua']
Novo participante: Metropolitanos
Novo participante: Red Bull Bragantino
Novo participante: Athletico Paranaense
Novo participante: Montevideo City Torque
Novo participante: 12 de Octubre
Copa de 2022:
Finalistas: ['Independiente del Valle', 'São Paulo']
Semis:  ['Atlético Goianiense', 'Melgar']
Quartas: ['Nacional'

# 2023-atual: 56 times - GP + PO

In [37]:
# 2023: f=0, g=8, po=20, b=21
# 2024: f=0, g=8, po=20, b=21
for i in range(2023, 2024+1):

  ano = str(i)
  df = pd.read_html('https://pt.wikipedia.org/wiki/Copa_Sul-Americana_de_' + ano)
  print(f'Copa de {i}:')
  g = 8
  p = 20
  f = 0
  b = 21
  dftemp = df[f].T.reset_index(drop=True)
  dftemp.columns = dftemp.iloc[0]
  dftemp = dftemp.drop(0)
  dftemp = dftemp.drop(2)
  campeao = clean_text(dftemp['Campeão'])
  vice = clean_text(dftemp['Vice-campeão'])
  finalistas = campeao.to_list() + vice.to_list()
  participantes, top_sf, top_qf, top_of = get_bracket(finalistas, df[b])

  top_po = get_times_preliminares(df[p], participantes)
  participantes = set(participantes).union(top_po)
  print(f'Playoff: {top_po}')

  top_gp = []
  for j in range(g, g+8):
    grupo = get_times_grupos(df[j], participantes)
    #top_gp = set(top_gp).union(grupo)
    top_gp = top_gp + grupo

  participantes = set(participantes).union(top_gp)
  print(f'Grupos: {list(top_gp)}')

  df_copa = preencher_tabela(participantes, df_copa)

Copa de 2023:
Finalistas: ['LDU Quito', 'Fortaleza']
Semis:  ['Corinthians', 'Defensa y Justicia']
Quartas: ['Estudiantes', 'América Mineiro', 'São Paulo', 'Botafogo']
Oitavas: ["Newell's Old Boys", 'Goiás', 'Red Bull Bragantino', 'Libertad', 'Ñublense', 'San Lorenzo', 'Guaraní', 'Emelec']
Playoff: ['Barcelona de Guayaquil', 'Patronato', 'Colo-Colo', 'Sporting Cristal', 'Independiente Medellín', 'Audax Italiano', 'Tigre', 'Universitario']
Grupos: ['Magallanes', 'Universidad César Vallejo', 'Danubio', 'Huracán', 'Tacuary', 'Oriente Petrolero', 'Deportes Tolima', 'Academia Puerto Cabello', 'Santos', 'Blooming', 'Millonarios', 'Peñarol', 'Santa Fe', 'Gimnasia y Esgrima', 'Palestino', 'Estudiantes de Mérida']
Novo participante: Academia Puerto Cabello
Novo participante: Magallanes
Novo participante: Patronato
Novo participante: Tacuary
Novo participante: América Mineiro
Copa de 2024:
Finalistas: ['Racing', 'Cruzeiro']
Semis:  ['Corinthians', 'Lanús']
Quartas: ['Fortaleza', 'Athletico Paran

# Corrigindo nomes dos times

In [38]:
# @title Participantes da Sulamericana
df_part = pd.read_html('https://pt.wikipedia.org/wiki/Participa%C3%A7%C3%B5es_de_clubes_brasileiros_de_futebol_em_competi%C3%A7%C3%B5es_internacionais#Copa_Sul-Americana')
part_sa = df_part[31]
part_sa_times = part_sa['Clube'].to_list()
part_sa_times

['São Paulo',
 'Fluminense',
 'Corinthians',
 'Atlético Mineiro',
 'Botafogo',
 'Goiás',
 'Santos',
 'Athletico Paranaense',
 'Bahia',
 'Internacional',
 'Vasco da Gama',
 'Cruzeiro',
 'Grêmio',
 'Flamengo',
 'Vitória',
 'Coritiba',
 'Palmeiras',
 'Sport',
 'Chapecoense',
 'Figueirense',
 'Atlético Goianiense',
 'Ceará',
 'Ponte Preta',
 'Cuiabá',
 'Fortaleza',
 'Red Bull Bragantino',
 'Criciúma',
 'Paraná',
 'São Caetano',
 'Juventude',
 'Avaí',
 'Grêmio Barueri',
 'Náutico',
 'Portuguesa',
 'Joinville',
 'Brasília',
 'Santa Cruz',
 'América Mineiro']

In [39]:
# @title Participantes da Copa Conmebol
part_cc_times = df_part[35]['Clube'].to_list()
part_cc_times

['Atlético Mineiro',
 'Bragantino',
 'Fluminense',
 'Botafogo',
 'Corinthians',
 'Grêmio',
 'Vasco',
 'Vitória',
 'América de Natal',
 'Ceará',
 'CSA',
 'Guarani',
 'Palmeiras',
 'Paraná',
 'Portuguesa',
 'Rio Branco',
 'Sampaio Corrêa',
 'Santos',
 'São Paulo',
 'São Raimundo',
 'Vila Nova']

In [40]:
lista_part_br = list(set(part_sa_times + part_cc_times))

In [41]:
substituicoes = {
    'Atlético Paranaense': 'Athletico Paranaense',
    'Bragantino' : 'Red Bull Bragantino',
    'Rio Branco' : 'Rio Branco-AC',
    'São Raimundo' : 'São Raimundo-AM'
}
df_copa['Time'] = df_copa['Time'].replace(substituicoes)
df_grouped = df_copa.groupby('Time', as_index=False)[colunas_df[1:]].sum()
len(df_grouped['Time'].sort_values().to_list())

231

In [42]:
df_copa = df_grouped.sort_values(by=colunas_df[1:], ascending=False).reset_index(drop=True)
df_copa

,Time,CON_B_1,CON_B_2,CON_B_SF,CON_B_QF,CON_B_OF,CON_B_PO,CON_B_GP,CON_B_F1,CON_B_F2,CON_B_F3,CON_B_P
0,Lanús,2,2,1,1,6,0,1,3,0,0,16
1,São Paulo,2,1,3,2,1,0,0,5,1,0,15
2,Atlético Mineiro,2,1,3,1,0,0,0,3,3,1,14
3,LDU Quito,2,1,2,2,5,0,1,1,0,0,14
4,Boca Juniors,2,0,1,2,3,0,0,2,0,0,10
...,...,...,...,...,...,...,...,...,...,...,...,...
226,Llaneros,0,0,0,0,0,0,0,0,1,0,1
227,Monagas,0,0,0,0,0,0,0,0,1,0,1
228,Mushuc Runa,0,0,0,0,0,0,0,0,1,0,1
229,Petrolero,0,0,0,0,0,0,0,0,1,0,1


In [43]:
df_copa = df_copa[df_copa['Time'].isin(lista_part_br)].reset_index(drop=True)
df_copa

,Time,CON_B_1,CON_B_2,CON_B_SF,CON_B_QF,CON_B_OF,CON_B_PO,CON_B_GP,CON_B_F1,CON_B_F2,CON_B_F3,CON_B_P
0,São Paulo,2,1,3,2,1,0,0,5,1,0,15
1,Atlético Mineiro,2,1,3,1,0,0,0,3,3,1,14
2,Athletico Paranaense,2,0,1,2,1,0,0,3,0,0,9
3,Internacional,1,0,1,2,1,1,0,1,1,0,8
4,Botafogo,1,0,0,3,5,0,0,2,0,0,11
5,Santos,1,0,0,3,2,0,1,2,1,0,10
6,Chapecoense,1,0,0,1,1,0,0,0,1,0,4
7,Fluminense,0,1,1,3,4,0,1,2,1,0,13
8,Red Bull Bragantino,0,1,0,1,4,0,0,0,0,0,6
9,Fortaleza,0,1,0,1,0,0,0,0,1,0,3


# Pontuação
Corrigir pontuação

| Resultado   | Pontos  | Pct %   |
|-------------|---------|---------|
| Campeão     | 2200.00 | 100.00% |
| Vice        |  766.67 |  33.33% |
| Semis       |  255.56 |  11.11% |
| Quartas     |  127.78 |   5.56% |
| Oitavas     |   76.67 |   3.33% |
| Playoff     |   28.60 |   1.30% |
| Fase Grupos |   25.91 |   1.18% |
| Pre -1      |    6.60 |   0.30% |
| Pre -2      |    3.30 |   0.15% |
| Pre -3      |    1.54 |   0.07% |

In [44]:
df_rank = df_copa.sort_values(by=colunas_df[1:], ascending=False).reset_index(drop=True)
df_rank.index += 1

In [45]:
df_rank.head(50)

,Time,CON_B_1,CON_B_2,CON_B_SF,CON_B_QF,CON_B_OF,CON_B_PO,CON_B_GP,CON_B_F1,CON_B_F2,CON_B_F3,CON_B_P
1,São Paulo,2,1,3,2,1,0,0,5,1,0,15
2,Atlético Mineiro,2,1,3,1,0,0,0,3,3,1,14
3,Athletico Paranaense,2,0,1,2,1,0,0,3,0,0,9
4,Internacional,1,0,1,2,1,1,0,1,1,0,8
5,Botafogo,1,0,0,3,5,0,0,2,0,0,11
6,Santos,1,0,0,3,2,0,1,2,1,0,10
7,Chapecoense,1,0,0,1,1,0,0,0,1,0,4
8,Fluminense,0,1,1,3,4,0,1,2,1,0,13
9,Red Bull Bragantino,0,1,0,1,4,0,0,0,0,0,6
10,Fortaleza,0,1,0,1,0,0,0,0,1,0,3


In [46]:
# Calculando a pontuação
titulo = 100
df_rank['PTS'] = round(
    df_rank['CON_B_1'] *1*titulo +
    df_rank['CON_B_2'] *0.3333*titulo +
    df_rank['CON_B_SF']*0.1111*titulo +
    df_rank['CON_B_QF']*0.0556*titulo +
    df_rank['CON_B_OF']*0.0333*titulo +
    df_rank['CON_B_PO']*0.0130*titulo +
    df_rank['CON_B_GP']*0.0118*titulo +
    df_rank['CON_B_F1']*0.0030*titulo +
    df_rank['CON_B_F2']*0.0015*titulo +
    df_rank['CON_B_F3']*0.0007*titulo, 2)

In [47]:
df_rank = df_rank.sort_values(by=['PTS', 'Time'], ascending=[False, True]).reset_index(drop=True)
df_rank.index += 1

# Ranking Geral

In [48]:
df_rank

,Time,CON_B_1,CON_B_2,CON_B_SF,CON_B_QF,CON_B_OF,CON_B_PO,CON_B_GP,CON_B_F1,CON_B_F2,CON_B_F3,CON_B_P,PTS
1,São Paulo,2,1,3,2,1,0,0,5,1,0,15,282.76
2,Atlético Mineiro,2,1,3,1,0,0,0,3,3,1,14,273.64
3,Athletico Paranaense,2,0,1,2,1,0,0,3,0,0,9,226.46
4,Botafogo,1,0,0,3,5,0,0,2,0,0,11,133.93
5,Internacional,1,0,1,2,1,1,0,1,1,0,8,127.31
6,Santos,1,0,0,3,2,0,1,2,1,0,10,125.27
7,Chapecoense,1,0,0,1,1,0,0,0,1,0,4,109.04
8,Fluminense,0,1,1,3,4,0,1,2,1,0,13,76.37
9,Corinthians,0,0,4,2,2,0,1,1,1,0,11,63.85
10,Red Bull Bragantino,0,1,0,1,4,0,0,0,0,0,6,52.21


# Ranking Top20

In [49]:
df_rank.head(20)

,Time,CON_B_1,CON_B_2,CON_B_SF,CON_B_QF,CON_B_OF,CON_B_PO,CON_B_GP,CON_B_F1,CON_B_F2,CON_B_F3,CON_B_P,PTS
1,São Paulo,2,1,3,2,1,0,0,5,1,0,15,282.76
2,Atlético Mineiro,2,1,3,1,0,0,0,3,3,1,14,273.64
3,Athletico Paranaense,2,0,1,2,1,0,0,3,0,0,9,226.46
4,Botafogo,1,0,0,3,5,0,0,2,0,0,11,133.93
5,Internacional,1,0,1,2,1,1,0,1,1,0,8,127.31
6,Santos,1,0,0,3,2,0,1,2,1,0,10,125.27
7,Chapecoense,1,0,0,1,1,0,0,0,1,0,4,109.04
8,Fluminense,0,1,1,3,4,0,1,2,1,0,13,76.37
9,Corinthians,0,0,4,2,2,0,1,1,1,0,11,63.85
10,Red Bull Bragantino,0,1,0,1,4,0,0,0,0,0,6,52.21


In [50]:
# Exportar para Excel
df_rank.to_excel('Sulamericana Rank.xlsx')